In [49]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import supervised_convnet
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import sys
from sklearn.model_selection import train_test_split


uncorrelated_data = np.load("../ising81x81_temp1_uncorrelated9x9.npy")[:10]
correlated_data = np.load("../ising81x81_temp1.npy")[:4000,:9,:9]
data = np.vstack((uncorrelated_data, correlated_data))
label = np.hstack((np.zeros(10), np.ones(4000)))
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.25, random_state=42)
# print(model(torch.Tensor(data[0]).unsqueeze(0).unsqueeze(1)))

In [50]:
y_train.shape

(3007,)

In [51]:
class IsingDataset(Dataset):
    def __init__(self, data, label):
        self.X = data
        self.y = label
        
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return len(self.X)
   



In [52]:
isingdataset = IsingDataset(X_train[:8000], y_train[:8000])
isingdataset.y

array([1., 1., 1., ..., 1., 1., 1.])

In [54]:
isingdataset.y.shape

(3007,)

In [106]:
model = supervised_convnet.SupervisedConvNet(filter_size = 3, square_size = 9)


In [108]:
# specify loss function
criterion = nn.BCELoss()

# specify loss function
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Create training and test dataloaders
num_workers = 0
# how many samples per batch to load
batch_size = 1000
# number of epochs to train the model
n_epochs = 300

# prepare data loaders
train_loader = torch.utils.data.DataLoader(isingdataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0

    ###################
    # train the model #
    ###################
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.unsqueeze(1).type('torch.FloatTensor')
        target = target.type('torch.FloatTensor')
        optimizer.zero_grad()
        output = model(data)[4].view(-1)
#         print(data[:10])
#         raise ValueError
        loss = criterion(output, target) 
        for param in model.parameters():
            loss += (((param)**2).sum())/20
#         print(loss)
        loss.backward()
        optimizer.step()

        # update running training loss
        train_loss += loss.item() * batch_size
    
    # print avg training statistics 
    train_loss = train_loss/len(train_loader)
    if epoch % 10 == 1:
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(
            epoch, 
            train_loss
            ))
#         print("data", data[:10])
        print("output", (output)[:10])
        print("target", (target)[:10])
#         for name, param in model.named_parameters():
#             if param.requires_grad:
#                 print (name, param.grad)

Epoch: 1 	Training Loss: 250.249963
output tensor([0.9663, 0.9781, 0.7723, 0.7723, 0.7723, 0.7723, 0.9781],
       grad_fn=<SliceBackward>)
target tensor([1., 1., 1., 1., 1., 1., 1.])
Epoch: 11 	Training Loss: 333.108764
output tensor([0.9255, 0.9712, 0.7739, 0.9726, 0.9726, 0.7739, 0.7739],
       grad_fn=<SliceBackward>)
target tensor([0., 1., 1., 1., 1., 1., 1.])
Epoch: 21 	Training Loss: 233.161747
output tensor([0.7723, 0.9770, 0.9770, 0.9770, 0.9770, 0.7723, 0.9770],
       grad_fn=<SliceBackward>)
target tensor([1., 1., 1., 1., 1., 1., 1.])
Epoch: 31 	Training Loss: 256.608702
output tensor([0.8058, 0.7746, 0.7746, 0.9773, 0.7746, 0.9773, 0.7746],
       grad_fn=<SliceBackward>)
target tensor([1., 1., 1., 1., 1., 1., 1.])
Epoch: 41 	Training Loss: 241.509635
output tensor([0.7746, 0.9776, 0.9776, 0.9776, 0.7746, 0.9776, 0.7746],
       grad_fn=<SliceBackward>)
target tensor([1., 1., 1., 1., 1., 1., 1.])
Epoch: 51 	Training Loss: 249.808807
output tensor([0.7731, 0.9783, 0.7731, 

In [ ]:
data[-1], output[-1], target[-1]

In [ ]:
model(data[-1].unsqueeze(0))

In [105]:
#initial
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)
        print ("sum", name, (param.data).sum())

conv2d.weight tensor([[[[0.1478, 0.1485, 0.1484],
          [0.1486, 0.1494, 0.1489],
          [0.1480, 0.1488, 0.1482]]]])
sum conv2d.weight tensor(1.3366)
conv2d.bias tensor([0.1503])
sum conv2d.bias tensor(0.1503)
linear.weight tensor([[0.1554, 0.1531, 0.1603, 0.1560, 0.1567, 0.1573, 0.1554, 0.1589, 0.1573]])
sum linear.weight tensor(1.4105)
linear.bias tensor([0.8835])
sum linear.bias tensor(0.8835)


In [ ]:
#initial
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)
        print ("sum", name, (param.data).sum())

In [ ]:
for _ in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.unsqueeze(1).type('torch.FloatTensor')[0].unsqueeze(1)
        print("data", data)
        target = target.type('torch.FloatTensor')
        optimizer.zero_grad()
        output = model(data)[-1].view(-1)
        print("output", output)
        print("target", target[0])
        loss = criterion(output, target[0])
        print("loss.data", loss.data)
        loss.backward()
        for name, param in model.named_parameters():
            if param.requires_grad:
                print (name, param.grad)
        optimizer.step()


In [ ]:
if 1:
    data = np.load("ising81x81_temp1.npy")[:, :3, :3]

    v = conv_autoencoder.ConvAutoencoder(3, 1)

    # specify loss function
    criterion = nn.MSELoss()

    # specify loss function
    optimizer = torch.optim.Adam(v.parameters(), lr=0.001)


    # Create training and test dataloaders
    num_workers = 0
    # how many samples per batch to load
    batch_size = 20

    # prepare data loaders
    train_loader = torch.utils.data.DataLoader(data[:8000], batch_size=batch_size, num_workers=num_workers)
    test_loader = torch.utils.data.DataLoader(data[:2000], batch_size=batch_size, num_workers=num_workers)

    # number of epochs to train the model
    n_epochs = 500
    l1_crit = nn.L1Loss(size_average=False)

    for epoch in range(1, n_epochs+1):
        # monitor training loss
        train_loss = 0.0
        
        ###################
        # train the model #
        ###################
        for d in train_loader:
            # no need to flatten images
            d = (d.unsqueeze(1)).type(torch.FloatTensor)
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            outputs = v(d).view(-1, 1, 3, 3)
            # calculate the loss
            loss = criterion(outputs, d)
    #         for param in v.parameters():
    #             loss += (torch.abs(param)).mean()/200
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update running training loss
            train_loss += loss.item() * batch_size
                
        # print avg training statistics 
        train_loss = train_loss/len(train_loader)
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(
            epoch, 
            train_loss
            ))

In [118]:
np.tanh(np.sum(-np.array([ 1.7783, -1.4541, -4.4681, -3.8083,  3.1658, -0.5841,  6.2371,  1.4833,
         -2.3994]) ) + 1.4065)

0.896872923564325